<a href="https://colab.research.google.com/github/shrishtinigam/dyslexic-character-recognition/blob/main/PCA_and_K_Fold_Cross_Val_Char_Recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building the dataset
### Getting the images from the drive folder

(Normal, Reversed) Larger CNN 1000 dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import string

In [ ]:
import cv2 as cv2
from google.colab.patches import cv2_imshow

## TARGET: Given a single image of a letter, the model is able to classify it as "Normal" or "Dyslexic" and able to tell character it is, as well as what it is supposed to be.

In [ ]:
alphabets = []
for letter in string.ascii_uppercase:
  alphabets.append(letter)

In [ ]:
images = []
x = 0
for letter in alphabets:
  image_paths = glob.glob(f"/content/drive/My Drive/TrainNormal20/" + letter + "*.png")
  for i in image_paths:
    image = cv2.imread(i)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    images.append([image, x])
  x += 1

In [ ]:
"""for letter in alphabets:
  image_paths_dyx = glob.glob(f"/content/drive/My Drive/TrainDyslexic20/" + letter + "*.png")
  for i in image_paths_dyx:
    image = cv2.imread(i)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    images.append([image, x])
  x += 1"""

'for letter in alphabets:\n  image_paths_dyx = glob.glob(f"/content/drive/My Drive/TrainDyslexic20/" + letter + "*.png")\n  for i in image_paths_dyx:\n    image = cv2.imread(i)\n    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n    images.append([image, x])\n  x += 1'

In [ ]:
x = 26
image_paths_dyx = glob.glob(f"/content/drive/My Drive/TrainDyslexic20/*.png")
for i in image_paths_dyx:
  image = cv2.imread(i)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  images.append([image, x])

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(images)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1042 non-null   object
 1   1       1042 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 16.4+ KB


In [ ]:
df.shape

(1042, 2)

In [ ]:
df

,0,1
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
...,...,...
1037,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",26
1038,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",26
1039,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",26
1040,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",26


In [ ]:
df.rename(columns = {0:'Character', 1:'Class'}, inplace = True)

In [ ]:
df.columns

Index(['Character', 'Class'], dtype='object')

In [ ]:
df = df.sample(frac=1)

In [ ]:
def zeroPadTill(image, r, c):
  ri, ci = image.shape
  # ri, ci, x = image.shape
  pad_rows = r-ri
  pad_cols = c-ci
  image = cv2.copyMakeBorder(image, pad_rows, 0, pad_cols, 0, cv2.BORDER_CONSTANT)
  return image

In [ ]:
data = []
for image in df['Character']:
  image = zeroPadTill(image, 32, 32)
  image = image.flatten()
  data.append(image)
data = np.array(data)

In [ ]:
for i in range(data.shape[0]):
  #print(data[i].shape)
  if(data[i].shape[0] != 32 or data[i].shape[1] != 32):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
labels = df['Class'].to_numpy()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
from sklearn.decomposition import PCA


import cv2
import os

import numpy as np

In [ ]:
data = pd.DataFrame(data)
labels = pd.DataFrame(labels)

In [ ]:
# define the larger model
def larger_model(input_shape):
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=input_shape, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
def training(x_train, x_test, y_train,  y_test, y_test_df, input_shape, fold):
  print("FOLD NO: ", fold)
  # build the model
  print("input shape", input_shape)
  model = larger_model(input_shape)
  # Fit the model
  model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=200)
  # Final evaluation of the model
  scores = model.evaluate(x_test, y_test, verbose=0)
  print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

  predicts = model.predict(x_test)

  predicts_labels = []
  for i in predicts:
    predicts_labels.append(np.argmax(i))
  predicts_labels = np.array(predicts_labels)



  y_test_df = np.array(y_test_df)

  from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score
  print('Precision: %.3f' % precision_score(y_test_df,predicts_labels,average='micro'))
  print('Recall: %.3f' % recall_score(y_test_df, predicts_labels,average='micro'))
  print('Accuracy: %.3f' % accuracy_score(y_test_df, predicts_labels))
  print('F1-Score: %.3f' % f1_score(y_test_df, predicts_labels,average='micro'))

  return accuracy_score(y_test_df, predicts_labels)

In [ ]:
import math
def closestDivisors(n):
    a = round(math.sqrt(n))
    while n%a > 0: a -= 1
    return a,n//a

In [ ]:
skf = KFold(n_splits = 10, random_state = 7, shuffle = True)
fold = 1
scores = []
for train_index, val_index in skf.split(np.zeros(len(data)), labels):
  training_data = data.iloc[train_index]
  training_labels = labels.iloc[train_index]
  validation_data = data.iloc[val_index]
  validation_labels = labels.iloc[val_index]



  np_training_data = training_data.to_numpy()
  #np_training_data = np.reshape(np_training_data, (np_training_data.shape[0], 32, 32))

  np_validation_data = validation_data.to_numpy()
  #np_validation_data = np.reshape(np_validation_data, (np_validation_data.shape[0], 32, 32))

  np_training_labels = training_labels.to_numpy()
  np_validation_labels = validation_labels.to_numpy()

  data_np = np.vstack((np_training_data, np_validation_data))
  labels_np = np.vstack((np_training_labels, np_validation_labels))

  pca = PCA(0.942)
  data_np_pca = pca.fit_transform(data_np)
  print(pca.explained_variance_ratio_)
  print(pca.n_components_)

  r, c = closestDivisors(pca.n_components_)
  print(r,c)
  if(r < 6 or c < 6):
    continue

  x_train, x_test, y_train, y_test = train_test_split(data_np_pca, labels_np, test_size=0.3,random_state =42)

  x_train = x_train.reshape((x_train.shape[0], r, c, 1)).astype('float32')
  x_test = x_test.reshape((x_test.shape[0], r, c, 1)).astype('float32')

  x_train = np.array(x_train) / 255
  x_test = np.array(x_test) / 255

  img_size = 32

  y_test_df = y_test.copy()

  y_train = np.array(y_train)
  y_test = np.array(y_test)

  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_classes = df["Class"].unique().size



  # X_pca.shape

  scores.append(training(x_train, x_test, y_train,  y_test, y_test_df, (r,c,1), fold))
  fold += 1

[0.07028247 0.06331392 0.05500946 0.04903478 0.03831073 0.03453505
 0.03290767 0.02823806 0.02506891 0.02444214 0.02205497 0.02103395
 0.02091311 0.01974121 0.01915328 0.017315   0.01624746 0.0151351
 0.01414199 0.01375712 0.01259448 0.0121061  0.01081763 0.01035144
 0.00953503 0.00925663 0.00899885 0.00830912 0.00822452 0.00763748
 0.00761062 0.00701223 0.00694361 0.00657874 0.00628694 0.00608748
 0.00569699 0.00561422 0.00548366 0.005344   0.00502707 0.00466896
 0.00456014 0.00444149 0.00440237 0.00426995 0.00416495 0.00403307
 0.00397105 0.00367785 0.00351003 0.00345147 0.00326903 0.00323571
 0.00318903 0.00317982 0.00306018 0.00287875 0.00279957 0.00271976
 0.00266548 0.00256627 0.00251081 0.00241849 0.00237476 0.00236363
 0.00232722 0.00226425 0.00221627 0.00215724 0.0020724  0.00201313
 0.00198702 0.00193312 0.0018714  0.00183649 0.0017955  0.0017574
 0.00173949 0.00164137 0.00162968 0.00157345 0.00156059 0.00149845
 0.00148048 0.00146605 0.00144418 0.00140567 0.0013908  0.001367

In [ ]:
scores

[0.5654952076677316,
 0.5686900958466453,
 0.5015974440894568,
 0.5143769968051118,
 0.5143769968051118,
 0.48242811501597443,
 0.5047923322683706,
 0.5559105431309904,
 0.5175718849840255,
 0.5431309904153354]